In [40]:
#!/usr/bin/env python3
import os
import json
import numpy as np
import trimesh
from pathlib import Path
from tqdm import tqdm
import pandas as pd

heat_maps_verts_colors = {}
json_heatmaps_dir = './public/data/'

with open("public/data/heat_maps.json") as f:
    elems = json.load(f)

discrete_points_normalized = {}

for elem in tqdm(elems):
    try:
        if elem["Type"] == "Surfaces":
            name = elem.get("RegionPath", "") + " " + elem.get("GroupName", "")
            name = name.replace("Heat Map", "").strip()
            file = json_heatmaps_dir + elem["URL"]
            with open(file) as f:
                data = json.load(f)
                heat_maps_verts_colors[name] = data['colors']
        elif elem["Type"] == "Glyph":
            RegionPath = elem.get("RegionPath", "")
            file = json_heatmaps_dir + elem["URL"]
            if "fre" in file and "Frequency" not in RegionPath:
                RegionPath += " Frequency"
                #print(RegionPath)
            if RegionPath not in discrete_points_normalized:
                with open(file) as f:
                    data = json.load(f)
                    pos = np.reshape(data["positions"]["0"], (-1, 3)).tolist()
                    #scale = np.reshape(data["scale"]["0"], (-1, 3))
                    #scale = scale[:,1].tolist()
                    discrete_points_normalized[RegionPath] = {
                        "positions": pos,
                        "colors": data["colors"]["0"],
                        #"scales": scale,
                    }
            file = os.path.basename(file)
            file = "public/data/Heat_maps/discrete_points_frequency/" + file.replace("nor", "fre").replace("_points", "_fre_points").replace("_pointys", "").replace("NFs", "NFS").replace("_fre_fre", "_fre")
            if not os.path.isfile(file):
                file = file.replace("_fre_points", "_fre")
            #print(file, os.path.isfile(file), RegionPath)
            if os.path.isfile(file) and "fre" in file and "Frequency" not in RegionPath:
                RegionPath += " Frequency"
                #print(RegionPath)
                if RegionPath not in discrete_points_normalized:
                    with open(file) as f:
                        data = json.load(f)
                        pos = np.reshape(data["positions"]["0"], (-1, 3)).tolist()
                        #scale = np.reshape(data["scale"]["0"], (-1, 3))
                        #scale = scale[:,1].tolist()
                        discrete_points_normalized[RegionPath] = {
                            "positions": pos,
                            "colors": data["colors"]["0"],
                            #"scales": scale,
                        }
    except Exception as e:
        print(f"{e} for {elem}")
        #raise

for code in tqdm(["Liei", "Lif", "Lii", "Laa", "Lam", "Lap", "Riei", "Rif", "Rii", "Raa", "Ram", "Rap"]):
    if code[0:2] == "Li":
        RegionPath = "Left Groin/Sub-Node Fields " + code
    elif code[0:2] == "Ri":
        RegionPath = "Right Groin/Sub-Node Fields " + code
    elif code[0:2] == "La":
        RegionPath = "Left Axilla/Sub-Node Fields " + code
    elif code[0:2] == "Ra":
        RegionPath = "Right Axilla/Sub-Node Fields " + code
    else:
        raise ValueError(f"Unknown code {code}")
    file = f"public/data/Heat_maps/Sub_fields/{code.lower()}_points_2.json"
    try:
        with open(file) as f:
            data = json.load(f)
            pos = np.reshape(data["positions"]["0"], (-1, 3)).tolist()
            discrete_points_normalized[RegionPath] = {
                "positions": pos,
                "colors": data["colors"]["0"],
            }
    except Exception as e:
        print(e)
    file = f"public/data/Heat_maps/discrete_points_frequency/{code.lower()}_fre_points_2.json"
    RegionPath += " Frequency"
    try:
        with open(file) as f:
            data = json.load(f)
            pos = np.reshape(data["positions"]["0"], (-1, 3)).tolist()
            discrete_points_normalized[RegionPath] = {
                "positions": pos,
                "colors": data["colors"]["0"],
            }
    except Exception as e:
        print(e)

print(f"Number of heatmaps: {len(heat_maps_verts_colors)}")
json.dump(heat_maps_verts_colors, open("heat_maps_verts_colors.json", 'w'))

nf = sum("Frequency" in k for k in discrete_points_normalized)
print(f"Number of frequency discrete points: {nf}/{len(discrete_points_normalized)}")

json.dump(discrete_points_normalized, open("discrete_points_normalized.json", 'w'))

print(sorted(discrete_points_normalized.keys()))
for k in heat_maps_verts_colors:
    if k not in discrete_points_normalized:
        print(f"Missing normalised for {k}")
    if k + " Frequency" not in discrete_points_normalized:
        print(f"Missing frequency for {k}")

  0%|                                                                                          | 0/133 [00:00<?, ?it/s]

[Errno 2] No such file or directory: './public/data/Heat_maps/discrete_points_normalized/lal_points_2.json' for {'GroupName': 'Patient Data', 'GlyphGeometriesURL': 'Heat_maps/discrete_points_normalized/lal_points_3.json', 'MorphColours': False, 'MorphNormals': False, 'MorphVertices': False, 'RegionPath': 'Left Axilla/Sub-Node Fields/Lal', 'Type': 'Glyph', 'URL': 'Heat_maps/discrete_points_normalized/lal_points_2.json'}


  2%|█▏                                                                                | 2/133 [00:00<00:41,  3.15it/s]

[Errno 2] No such file or directory: './public/data/Heat_maps/discrete_points_normalized/ral_points_2.json' for {'GroupName': 'Patient Data', 'GlyphGeometriesURL': 'Heat_maps/discrete_points_normalized/ral_points_3.json', 'MorphColours': False, 'MorphNormals': False, 'MorphVertices': False, 'RegionPath': 'Right Axilla/Sub-Node Fields/Ral', 'Type': 'Glyph', 'URL': 'Heat_maps/discrete_points_normalized/ral_points_2.json'}


 26%|████████████████████▋                                                            | 34/133 [00:18<01:00,  1.65it/s]

...

 33%|███████████████████████████▋                                                       | 4/12 [00:00<00:00, 37.22it/s]

[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/liei_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/lif_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/lii_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/laa_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/lam_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/lap_points_2.json'


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 35.37it/s]

[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/riei_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/rif_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/rii_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/raa_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/ram_points_2.json'
[Errno 2] No such file or directory: 'public/data/Heat_maps/Sub_fields/rap_points_2.json'
Number of heatmaps: 50


Number of frequency discrete points: 41/77
['1 Draining Node Fields', '1 Draining Node Fields Frequency', '2 Or More Draining Node Fields', '2 Or More Draining Node Fields Frequency', '3 Or More Draining Node Fields', '3 Or More Draining Node Fields Frequency', '4 Or More Draining Node Fields', '4 Or More Draining Node Fields Frequency', 'All Patient Data Frequency', 'Anterior Head', 'Left Axilla', 'Left Axilla/Sub-Node Fields Laa Frequency', 'Left Axilla/Sub-Node Fields Lam Frequency', 'Left Axilla/Sub-Node Fields Lap Frequency', 'Left Cervical Level I', 'Left Cervical Level I Frequency', 'Left Cervical Level II', 'Left Cervical Level II Frequency', 'Left Cervical Level III', 'Left Cervical Level III Frequency', 'Left Cervical Level IV', 'Left Cervical Level IV Frequency', 'Left Cervical Level V', 'Left Cervical Level V Frequency', 'Left Epitrochlear', 'Left Epitrochlear Frequency', 'Left Groin', 'Left Groin/Sub-Node Fields Liei Frequency', 'Left Groin/Sub-Node Fields Lif Frequency', 

In [42]:
#!/usr/bin/env python3 

import os 

import json 

import numpy as np 

import trimesh 

from pathlib import Path 

from tqdm import tqdm 

import pandas as pd 

  

heat_maps_verts_colors = {} 

json_heatmaps_dir = './public/data/' 

  

with open("public/data/heat_maps.json") as f: 

    elems = json.load(f) 

  

discrete_points_normalized = {} 

  

for elem in tqdm(elems): 

    try: 

        if elem["Type"] == "Surfaces": 

            name = elem.get("RegionPath", "") + " " + elem.get("GroupName", "") 

            name = name.replace("Heat Map", "").strip() 

            file = json_heatmaps_dir + elem["URL"] 

            with open(file) as f: 

                data = json.load(f) 

                heat_maps_verts_colors[name] = data['colors'] 

        elif elem["Type"] == "Glyph": 

            RegionPath = elem.get("RegionPath", "") 

            RegionPath = RegionPath.replace("Sub-Node Fields/", "Sub-Node Fields ") 

            file = json_heatmaps_dir + elem["URL"] 

            if "fre" in file and "Frequency" not in RegionPath: 

                RegionPath += " Frequency" 

                #print(RegionPath) 

            if RegionPath not in discrete_points_normalized: 

                with open(file) as f: 

                    data = json.load(f) 

                    pos = np.reshape(data["positions"]["0"], (-1, 3)).tolist() 

                    #scale = np.reshape(data["scale"]["0"], (-1, 3)) 

                    #scale = scale[:,1].tolist() 

                    discrete_points_normalized[RegionPath] = { 

                        "positions": pos, 

                        "colors": data["colors"]["0"] if "colors" in data else [], 

                        #"scales": scale, 

                    } 

            file = os.path.basename(file) 

            file = "public/data/Heat_maps/discrete_points_frequency/" + file.replace("nor", "fre").replace("_points", "_fre_points").replace("_pointys", "").replace("NFs", "NFS").replace("_fre_fre", "_fre") 

            if not os.path.isfile(file): 

                file = file.replace("_fre_points", "_fre") 

            #print(file, os.path.isfile(file), RegionPath) 

            if os.path.isfile(file) and "fre" in file and "Frequency" not in RegionPath: 

                RegionPath += " Frequency" 

                #print(RegionPath) 

                if RegionPath not in discrete_points_normalized: 

                    with open(file) as f: 

                        data = json.load(f) 

                        pos = np.reshape(data["positions"]["0"], (-1, 3)).tolist() 

                        #scale = np.reshape(data["scale"]["0"], (-1, 3)) 

                        #scale = scale[:,1].tolist() 

                        discrete_points_normalized[RegionPath] = { 

                            "positions": pos, 

                            "colors": data["colors"]["0"] if "colors" in data else [], 

                            #"scales": scale, 

                        } 

    except Exception as e: 

        print(f"{e} for {elem}") 

        #raise 

  

for code in tqdm(["Liei", "Lif", "Lii", "Laa", "Lam", "Lap", "Lal", "Riei", "Rif", "Rii", "Raa", "Ram", "Rap", "Ral"]): 

    if code[0:2] == "Li": 

        RegionPath = "Left Groin/Sub-Node Fields " + code 

    elif code[0:2] == "Ri": 

        RegionPath = "Right Groin/Sub-Node Fields " + code 

    elif code[0:2] == "La": 

        RegionPath = "Left Axilla/Sub-Node Fields " + code 

    elif code[0:2] == "Ra": 

        RegionPath = "Right Axilla/Sub-Node Fields " + code 

    else: 

        raise ValueError(f"Unknown code {code}") 

    file = f"public/data/Heat_maps/Sub_fields/Normalized/{code.lower()}_points_2.json" 

    try: 

        with open(file) as f: 

            data = json.load(f) 

            pos = np.reshape(data["positions"]["0"], (-1, 3)).tolist() 

            discrete_points_normalized[RegionPath] = { 

                "positions": pos, 

                "colors": data["colors"]["0"], 

            } 

    except Exception as e: 

        print(e) 

    file = f"public/data/Heat_maps/Sub_fields/frequency/{code.lower()}_fre_points_2.json" 

    RegionPath += " Frequency" 

    try: 

        with open(file) as f: 

            data = json.load(f) 

            pos = np.reshape(data["positions"]["0"], (-1, 3)).tolist() 

            discrete_points_normalized[RegionPath] = { 

                "positions": pos, 

                

            } 

    except Exception as e: 

        print(e) 

  

print(f"Number of heatmaps: {len(heat_maps_verts_colors)}") 

json.dump(heat_maps_verts_colors, open("heat_maps_verts_colors.json", 'w')) 

  

nf = sum("Frequency" in k for k in discrete_points_normalized) 

print(f"Number of frequency discrete points: {nf}/{len(discrete_points_normalized)}") 

  

json.dump(discrete_points_normalized, open("discrete_points_normalized.json", 'w')) 

  

print(sorted(discrete_points_normalized.keys())) 

for k in heat_maps_verts_colors: 

    if k not in discrete_points_normalized: 

        print(f"Missing normalised for {k}") 

    if k + " Frequency" not in discrete_points_normalized: 

        print(f"Missing frequency for {k}") 

  0%|                                                                                          | 0/133 [00:00<?, ?it/s]

[Errno 2] No such file or directory: './public/data/Heat_maps/discrete_points_normalized/lal_points_2.json' for {'GroupName': 'Patient Data', 'GlyphGeometriesURL': 'Heat_maps/discrete_points_normalized/lal_points_3.json', 'MorphColours': False, 'MorphNormals': False, 'MorphVertices': False, 'RegionPath': 'Left Axilla/Sub-Node Fields/Lal', 'Type': 'Glyph', 'URL': 'Heat_maps/discrete_points_normalized/lal_points_2.json'}


  2%|█▏                                                                                | 2/133 [00:00<00:31,  4.21it/s]

[Errno 2] No such file or directory: './public/data/Heat_maps/discrete_points_normalized/ral_points_2.json' for {'GroupName': 'Patient Data', 'GlyphGeometriesURL': 'Heat_maps/discrete_points_normalized/ral_points_3.json', 'MorphColours': False, 'MorphNormals': False, 'MorphVertices': False, 'RegionPath': 'Right Axilla/Sub-Node Fields/Ral', 'Type': 'Glyph', 'URL': 'Heat_maps/discrete_points_normalized/ral_points_2.json'}


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 20.09it/s]


Number of heatmaps: 50
Number of frequency discrete points: 51/101
['1 Draining Node Fields', '1 Draining Node Fields Frequency', '2 Or More Draining Node Fields', '2 Or More Draining Node Fields Frequency', '3 Or More Draining Node Fields', '3 Or More Draining Node Fields Frequency', '4 Or More Draining Node Fields', '4 Or More Draining Node Fields Frequency', 'All Patient Data Frequency', 'Anterior Head', 'Anterior Head Frequency', 'Left Axilla', 'Left Axilla Frequency', 'Left Axilla/Sub-Node Fields Laa', 'Left Axilla/Sub-Node Fields Laa Frequency', 'Left Axilla/Sub-Node Fields Lal', 'Left Axilla/Sub-Node Fields Lal Frequency', 'Left Axilla/Sub-Node Fields Lam', 'Left Axilla/Sub-Node Fields Lam Frequency', 'Left Axilla/Sub-Node Fields Lap', 'Left Axilla/Sub-Node Fields Lap Frequency', 'Left Cervical Level I', 'Left Cervical Level I Frequency', 'Left Cervical Level II', 'Left Cervical Level II Frequency', 'Left Cervical Level III', 'Left Cervical Level III Frequency', 'Left Cervical L